In [7]:
from pyspark.sql.functions import when, col, regexp_replace, trim
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, explode, explode_outer, split
from pyspark.sql.types import StringType, IntegerType, StructType, StructField
from pyspark.sql.functions import col
from pyspark.sql.types import ArrayType,StructType
import json

import pyspark
from pyspark import SparkConf, SparkContext
from delta import *
from pyspark.sql.functions import col, when, md5, concat_ws, lit
from pyspark.sql.types import StringType, StructType, StructField, TimestampType
from pyspark.sql.functions import *
from datetime import datetime
 
builder = pyspark.sql.SparkSession.builder.appName("ReadDelta") \
.master("local[*]") \
.config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0") \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
 
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [8]:
%run "/home/jovyan/work/notebooks/1.ev_analyze_extract_data.ipynb"

root
 |-- data: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- meta: struct (nullable = true)
 |    |-- view: struct (nullable = true)
 |    |    |-- approvals: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- reviewedAt: long (nullable = true)
 |    |    |    |    |-- reviewedAutomatically: boolean (nullable = true)
 |    |    |    |    |-- state: string (nullable = true)
 |    |    |    |    |-- submissionDetails: struct (nullable = true)
 |    |    |    |    |    |-- permissionType: string (nullable = true)
 |    |    |    |    |-- submissionId: long (nullable = true)
 |    |    |    |    |-- submissionObject: string (nullable = true)
 |    |    |    |    |-- submissionOutcome: string (nullable = true)
 |    |    |    |    |-- submissionOutcomeApplication: struct (nullable = true)
 |    |    |    |    |    |-- failureCount: long (nullable = true

In [9]:
%run "/home/jovyan/work/notebooks/2.populate_bronze.ipynb"

+----------+-------------+-----------+--------------------+--------+
|Table_Name|     Progress|     Status|          Start_time|End_time|
+----------+-------------+-----------+--------------------+--------+
|   EV_Data|Ingest_Bronze|In_Progress|2024-12-06 19:20:...|    null|
+----------+-------------+-----------+--------------------+--------+

+----------+----------+-----------+--------+
|Layer_Name|Table_Name|Schema_Flag|Md5_Hash|
+----------+----------+-----------+--------+
+----------+----------+-----------+--------+

+----------+----------+-----------+--------------------+
|Layer_Name|Table_Name|Schema_Flag|            Md5_Hash|
+----------+----------+-----------+--------------------+
|    Bronze|   EV_Data|          N|99b6b6813c00543c7...|
+----------+----------+-----------+--------------------+

check_and_store_md5_result-True
+------------------+--------------------+--------+----------+------------+----------+------------+----+----------+-----------+-----------+-----+-----------

DataFrame[sid: string, id: string, position: string, created_at: string, created_meta: string, updated_at: string, updated_meta: string, meta: string, VIN_1_10: string, County: string, City: string, State: string, Postal_Code: string, Model_Year: string, Make: string, Model: string, Electric_Vehicle_Type: string, Clean_Alternative_Fuel_Vehicle_CAFV_Eligibility: string, Electric_Range: string, Base_MSRP: string, Legislative_District: string, DOL_Vehicle_ID: string, Vehicle_Location: string, Electric_Utility: string, 2020_Census_Tract: string, Counties: string, Congressional_Districts: string, WAOFM_GIS_Legislative_District_Boundary: string]

+----------+-------------+-----------+--------------------+--------+
|Table_Name|     Progress|     Status|          Start_time|End_time|
+----------+-------------+-----------+--------------------+--------+
|   EV_Data|Ingest_Bronze|In_Progress|2024-12-06 19:20:...|    null|
+----------+-------------+-----------+--------------------+--------+

+----------+-------------+-----------+--------------------+--------+
|Table_Name|     Progress|     Status|          Start_time|End_time|
+----------+-------------+-----------+--------------------+--------+
|   EV_Data|Ingest_Bronze|In_Progress|2024-12-06 19:20:...|    null|
+----------+-------------+-----------+--------------------+--------+

+----------+-------------+---------+--------------------+--------------------+
|Table_Name|     Progress|   Status|          Start_time|            End_time|
+----------+-------------+---------+--------------------+--------------------+
|   EV_Data|Ingest_Bronze|Completed|2024-12-06 19:20:...|2024-12-06 19:

In [10]:
%run "/home/jovyan/work/notebooks/3.populate_silver.ipynb"

+----------+-------------+-----------+--------------------+--------+
|Table_Name|     Progress|     Status|          Start_time|End_time|
+----------+-------------+-----------+--------------------+--------+
|   EV_Data|Ingest_Silver|In_Progress|2024-12-06 19:23:...|    null|
+----------+-------------+-----------+--------------------+--------+

+------------+------------+
|        name|dataTypeName|
+------------+------------+
|         sid|   meta_data|
|          id|   meta_data|
|    position|   meta_data|
|  created_at|   meta_data|
|created_meta|   meta_data|
|  updated_at|   meta_data|
|updated_meta|   meta_data|
|        meta|   meta_data|
|  VIN (1-10)|        text|
|      County|        text|
+------------+------------+

+------------+------------+------------+
|        name|dataTypeName|SQL_DataType|
+------------+------------+------------+
|         sid|   meta_data|      STRING|
|          id|   meta_data|      STRING|
|    position|   meta_data|      STRING|
|  created_at| 

In [11]:
%run "/home/jovyan/work/notebooks/4.populate_gold.ipynb"

+----------+-----------+-----------+--------------------+--------+
|Table_Name|   Progress|     Status|          Start_time|End_time|
+----------+-----------+-----------+--------------------+--------+
|   EV_Data|Ingest_Gold|In_Progress|2024-12-06 19:24:...|    null|
+----------+-----------+-----------+--------------------+--------+

+------------------+--------------------+--------+-------------------+------------+-------------------+------------+----+----------+-----------+-----------+-----+-----------+----------+---------+-------+---------------------+-----------------------------------------------+--------------+---------+--------------------+--------------+--------------------+--------------------+-----------------+--------+-----------------------+---------------------------------------+--------+-----------+-------+
|               sid|                  id|position|         created_at|created_meta|         updated_at|updated_meta|meta|  VIN_1_10|     County|       City|State|Posta